In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.optim as optim
import BatchMaker

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import PIL
import random
import time
import os
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.0
Torchvision Version:  0.6.0


In [32]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(9, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
    
        self.layer2 = nn.Sequential( 
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
        
        self.layer3 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer4 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer6 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer10 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer11 = nn.Sequential(
            nn.Conv2d(256, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer12 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer13 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer14 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer15 = nn.Sequential(
            nn.Conv2d(128, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer16 = nn.Sequential(
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer17 = nn.Sequential(
            nn.Conv2d(64, 1,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(1))
            
            
    def forward(self,x):
        x = self.layer1(x)  
        x = self.layer2(x)  
        x = self.layer3(x)  
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)  
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)  
        x = self.layer10(x)  
        x = self.layer11(x)  
        x = self.layer12(x)  
        x = self.layer13(x)  
        x = self.layer14(x)  
        x = self.layer15(x)  
        x = self.layer16(x)
        x = self.layer17(x) 

        return x

In [33]:
# Check if GPU is available and move the model over to GPU
if torch.cuda.is_available():
    device = torch.device("cuda: 0")
    gpu_name = torch.cuda.get_device_name()
    print(f"Running on your {gpu_name} (GPU)")
else:
    device = torch.device("cpu")
    print("Running on your CPU")

net = CNN().to(device)

Running on your GeForce GTX 1080 (GPU)


In [37]:
Generator = BatchMaker.BatchMaker

batch_size = 1
training_file = "training.csv"
testing_file = "testing.csv"

In [35]:
loss_fn = nn.L1Loss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=0.001)

epochs = 1
epochs_no_improve_limit = 7

train_losses = []
val_losses = []


def train():
    epochs_no_improve = 0
    min_val_loss = np.Inf
    since = time.time()
    iteration = 0
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)
        val_loss = 0
        train_loss = 0
        
        for train_inputs, train_labels in Generator(training_file, batch_size):
            optimizer.zero_grad()
            output = net(torch.from_numpy(train_inputs.transpose(0,3,1,2)).to(device))
            loss = loss_fn(output[0].cpu(), torch.from_numpy(train_labels).cpu())
            train_loss += loss
            
#             fig = plt.figure(figsize = (32,24))
#             ax1 = fig.add_subplot(2, 2, 1)
#             ax1.set_title('Input red')
#             ax2 = fig.add_subplot(2, 2, 2)
#             ax2.set_title('Input green')
#             ax3 = fig.add_subplot(2, 2, 3)
#             ax3.set_title('Label')
#             ax4 = fig.add_subplot(2, 2, 4)
#             ax4.set_title('Output of the CNN')

#             sns.heatmap(train_inputs[0][0], vmin=0, vmax=1, cmap='gray', ax=ax1, cbar_kws={'shrink': .9})
#             sns.heatmap(train_inputs[0][1], vmin=0, vmax=1, ax=ax2, cbar_kws={'shrink': .9})
#             sns.heatmap(train_labels[0][0], vmin=0, vmax=1, ax=ax3, cbar_kws={'shrink': .9})
#             sns.heatmap(output.cpu().detach().numpy()[0], vmin=0, vmax=1, ax=ax4, cbar_kws={'shrink': .9})
#             plt.show()


#             writer.add_scalar('Loss/train', loss, iteration)

            print('Training Loss: {:.4f}'.format(loss.item()))
            loss.backward()
            optimizer.step()
            iteration += 1
        
        train_loss = train_loss / len(dataloaders['train'])
        train_losses.append(float(train_loss))
        
        del train_inputs
        del train_labels
        del output
        del loss
        torch.cuda.empty_cache()
        
#         with torch.no_grad():
#             for val_inputs, val_labels in dataloaders['val']:
#                 torch.cuda.empty_cache()
#                 output = net(val_inputs.to(device)).reshape(batch_size, output_height, output_width)
#                 loss = loss_fn(output.cpu(), val_labels.cpu().reshape(batch_size, output_height, output_width))
#                 val_loss += loss

#             val_loss = val_loss / len(dataloaders['val'])
#             val_losses.append(float(val_loss))
#             print('-' * 10)
#             print('Validation Loss: {:.4f}'.format(val_loss))

#             if val_loss < min_val_loss:
#                 torch.save({'state_dict': net.state_dict()}, 'Nets/pt-labi_CNN.pt')
#                 epochs_no_improve = 0
#                 min_val_loss = val_loss
#             else:
#                 epochs_no_improve += 1
#                 if epochs_no_improve == epochs_no_improve_limit:
#                     print('Early stopping initiated')
#                     model = torch.load('Nets/pt-labi_CNN.pt')
#                     print('Best model so far has been loaded')
    print('Least validation Loss: {:4f}'.format(min_val_loss))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Finished training')
train()

Epoch 1/1
----------
Training Loss: 34086.1683
Training Loss: 27450.4150


C:\Users\elias\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:88: UserWarning: Using a target size (torch.Size([1, 180, 240])) that is different to the input size (torch.Size([1, 1, 180, 240])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Training Loss: 15299.3694
Training Loss: 9004.5104
Training Loss: 8671.5615
Training Loss: 8301.8524
Training Loss: 10481.1345
Training Loss: 6925.1842
Training Loss: 5357.2153
Training Loss: 5273.2144
Training Loss: 4573.9428
Training Loss: 4221.2831
Training Loss: 3880.0721
Training Loss: 3291.6574
Training Loss: 3234.1603
Training Loss: 3230.5309
Training Loss: 3899.7386
Training Loss: 2800.9797
Training Loss: 5183.7385
Training Loss: 2359.2413
Training Loss: 2601.3941
Training Loss: 2475.3319
Training Loss: 4396.9612
Training Loss: 4294.7806
Training Loss: 2715.8656
Training Loss: 1973.0419
Training Loss: 2597.4471
Training Loss: 3982.0544
Training Loss: 3083.3913
Training Loss: 2616.2866
Training Loss: 3402.7675
Training Loss: 2703.0968
Training Loss: 1451.6043
Training Loss: 1456.2591
Training Loss: 1520.7868
Training Loss: 1569.3016
Training Loss: 7053.5138
Training Loss: 2057.8034
Training Loss: 2715.5516
Training Loss: 1933.2073
Training Loss: 2234.7379
Training Loss: 2220.714

Training Loss: 79.8274
Training Loss: 76.3976
Training Loss: 68.8154
Training Loss: 66.0646
Training Loss: 71.7349
Training Loss: 72.4872
Training Loss: 68.8141
Training Loss: 62.4835
Training Loss: 64.9922
Training Loss: 62.7572
Training Loss: 67.2876
Training Loss: 68.6978
Training Loss: 65.6196
Training Loss: 65.1834
Training Loss: 67.2029
Training Loss: 64.5304
Training Loss: 65.9276
Training Loss: 67.6866
Training Loss: 64.9250
Training Loss: 65.5971
Training Loss: 67.3562
Training Loss: 64.4523
Training Loss: 66.2067
Training Loss: 68.1504
Training Loss: 65.5576
Training Loss: 64.7970
Training Loss: 66.4176
Training Loss: 63.3915
Training Loss: 67.3558
Training Loss: 69.3970
Training Loss: 66.8944
Training Loss: 63.3531
Training Loss: 64.8996
Training Loss: 61.8217
Training Loss: 63.4377
Training Loss: 63.1869
Training Loss: 62.1162
Training Loss: 64.4277
Training Loss: 62.7082
Training Loss: 66.8884
Training Loss: 67.7908
Training Loss: 64.1243
Training Loss: 67.1961
Training Lo

Training Loss: 64.6031
Training Loss: 63.1345
Training Loss: 66.1580
Training Loss: 66.9644
Training Loss: 63.4209
Training Loss: 67.7648
Training Loss: 70.1769
Training Loss: 67.9374
Training Loss: 62.1146
Training Loss: 63.5354
Training Loss: 63.1067
Training Loss: 62.1933
Training Loss: 64.2630
Training Loss: 62.4014
Training Loss: 67.2258
Training Loss: 68.3456
Training Loss: 65.0854
Training Loss: 65.8127
Training Loss: 67.9916
Training Loss: 65.5437
Training Loss: 64.6528
Training Loss: 66.2398
Training Loss: 63.4008
Training Loss: 67.1391
Training Loss: 68.9714
Training Loss: 66.2122
Training Loss: 64.2702
Training Loss: 66.1054
Training Loss: 63.4903
Training Loss: 66.8426
Training Loss: 68.4907
Training Loss: 65.5662
Training Loss: 65.0514
Training Loss: 67.0183
Training Loss: 64.5224
Training Loss: 65.6840
Training Loss: 67.2340
Training Loss: 64.2219
Training Loss: 66.4509
Training Loss: 68.4877
Training Loss: 66.0552
Training Loss: 64.0674
Training Loss: 65.5652
Training Lo

Training Loss: 63.1166
Training Loss: 65.7437
Training Loss: 66.1863
Training Loss: 62.3402
Training Loss: 69.1150
Training Loss: 71.7791
Training Loss: 69.7920
Training Loss: 63.6189
Training Loss: 69.8212
Training Loss: 74.3788
Training Loss: 74.2366
Training Loss: 69.8645
Training Loss: 61.9480
Training Loss: 65.0452
Training Loss: 63.4482
Training Loss: 65.9370
Training Loss: 66.8480
Training Loss: 63.4240
Training Loss: 67.6266
Training Loss: 69.9433
Training Loss: 67.6441
Training Loss: 62.4304
Training Loss: 63.9010
Training Loss: 62.6641
Training Loss: 62.6661
Training Loss: 63.7229
Training Loss: 61.8334
Training Loss: 67.7843
Training Loss: 68.9285
Training Loss: 65.7149
Training Loss: 65.1052
Training Loss: 67.2487
Training Loss: 64.7943
Training Loss: 65.3712
Training Loss: 66.9656
Training Loss: 64.1575
Training Loss: 66.3170
Training Loss: 68.1268
Training Loss: 65.3722
Training Loss: 65.0681
Training Loss: 66.9017
Training Loss: 64.3091
Training Loss: 65.9655
Training Lo

Training Loss: 62.2492
Training Loss: 66.1709
Training Loss: 66.2415
Training Loss: 62.0721
Training Loss: 69.6717
Training Loss: 72.6035
Training Loss: 70.8690
Training Loss: 64.9351
Training Loss: 68.3031
Training Loss: 72.6711
Training Loss: 72.3697
Training Loss: 67.8656
Training Loss: 64.0890
Training Loss: 67.2963
Training Loss: 65.8100
Training Loss: 63.5040
Training Loss: 64.3277
Training Loss: 62.8110
Training Loss: 61.9964
Training Loss: 64.8729
Training Loss: 63.4171
Training Loss: 65.8312
Training Loss: 66.6307
Training Loss: 63.1178
Training Loss: 68.0078
Training Loss: 70.3992
Training Loss: 68.1788
Training Loss: 61.8232
Training Loss: 63.2319
Training Loss: 63.3894
Training Loss: 61.9012
Training Loss: 64.5360
Training Loss: 62.6901
Training Loss: 66.8916
Training Loss: 68.0019
Training Loss: 64.7690
Training Loss: 66.0730
Training Loss: 68.2338
Training Loss: 65.8062
Training Loss: 64.3406
Training Loss: 65.9145
Training Loss: 63.0989
Training Loss: 67.3887
Training Lo

Training Loss: 65.1858
Training Loss: 63.1682
Training Loss: 63.1430
Training Loss: 64.7858
Training Loss: 64.3064
Training Loss: 64.0857
Training Loss: 64.1075
Training Loss: 63.7623
Training Loss: 63.2442
Training Loss: 65.1650
Training Loss: 65.2177
Training Loss: 62.6055
Training Loss: 62.0620
Training Loss: 66.3505
Training Loss: 66.4235
Training Loss: 62.2626
Training Loss: 69.4637
Training Loss: 72.3891
Training Loss: 70.6556
Training Loss: 64.7291
Training Loss: 68.4927
Training Loss: 72.8554
Training Loss: 72.5554
Training Loss: 68.0591
Training Loss: 63.8789
Training Loss: 67.0806
Training Loss: 65.5958
Training Loss: 63.7070
Training Loss: 64.5301
Training Loss: 62.5995
Training Loss: 62.2033
Training Loss: 64.6575
Training Loss: 63.2034
Training Loss: 66.0335
Training Loss: 66.8321
Training Loss: 63.3246
Training Loss: 67.7863
Training Loss: 70.1739
Training Loss: 67.9566
Training Loss: 62.0329
Training Loss: 63.4397
Training Loss: 63.1736
Training Loss: 62.1112
Training Lo

Training Loss: 70.2136
Training Loss: 63.0786
Training Loss: 67.5347
Training Loss: 67.1823
Training Loss: 62.5025
Training Loss: 69.5758
Training Loss: 72.9453
Training Loss: 71.7551
Training Loss: 66.4609
Training Loss: 66.2295
Training Loss: 70.0884
Training Loss: 69.1988
Training Loss: 64.0355
Training Loss: 68.4956
Training Loss: 72.2505
Training Loss: 71.4070
Training Loss: 66.4251
Training Loss: 65.9802
Training Loss: 69.5821
Training Loss: 68.4612
Training Loss: 63.0898
Training Loss: 69.6102
Training Loss: 73.5309
Training Loss: 72.8367
Training Loss: 67.9892
Training Loss: 64.2673
Training Loss: 67.7585
Training Loss: 66.5381
Training Loss: 62.5422


KeyboardInterrupt: 

In [38]:
output_width = 240
output_height = 180
def test():
    test_loss = 0
    num_test_samples = 10
    with torch.no_grad():
        for idx, (test_inputs, test_labels) in enumerate(Generator(testing_file, 1)):
            if idx >= num_test_samples:
                break
            else:
                output = net(torch.from_numpy(test_inputs.transpose(0,3,1,2)).to(device))
                loss = loss_fn(output[0].cpu(), torch.from_numpy(test_labels).cpu())
                output = output[0].reshape(output_height, output_width).cpu().detach().numpy()
                test_labels = test_labels[0].reshape(output_height, output_width)
                test_loss += loss

                fig = plt.figure(figsize = (32,24))
                ax1 = fig.add_subplot(2, 2, 1)
                ax1.set_title('Output')
                ax2 = fig.add_subplot(2, 2, 2)
                ax2.set_title('Label')

                sns.heatmap(output, vmin=0, vmax=1, ax=ax1, square=True, cbar_kws={'shrink': .8})
                sns.heatmap(test_labels, vmin=0, vmax=1, ax=ax2, square=True, cbar_kws={'shrink': .8})
                plt.show()
        
        test_loss = test_loss/num_test_samples
        print('Average test loss: ' ,test_loss.numpy())
        print('Testing completed')
test()

FileNotFoundError: [Errno 2] No such file or directory: 'testing.csv'